In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [131]:
image = cv2.imread('new2.png',1)
image = cv2.resize(image, (640,480), interpolation = cv2.INTER_AREA)

In [128]:
blurred = cv2.pyrMeanShiftFiltering(image,31,91)
gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

In [114]:
retval, thresh = cv2.threshold(gray,242,255,cv2.THRESH_BINARY_INV)
im2,contours,hie = cv2.findContours(thresh,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

In [116]:
#cnt = contours[1:]
z = cv2.drawContours(image,[c1,c2,c3],0,(0,0,255),4)

In [133]:
cv2.imshow("im",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [66]:
len(contours)

15

In [132]:
x,y,w,h = cv2.boundingRect(c3)
img = cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)

In [119]:
rect = cv2.minAreaRect(c1)
box = cv2.boxPoints(rect)
box = np.int0(box)
im = cv2.drawContours(image,[box],0,(0,0,255),2)

In [83]:
perimeter = cv2.arcLength(contours[2],True)

In [151]:
rect

((267.2761535644531, 387.0914001464844),
 (138.1724853515625, 46.04342269897461),
 -13.840694427490234)

In [144]:
rows,cols = image.shape[:2]
[vx,vy,x,y] = cv2.fitLine(c3, cv2.DIST_L2,0,0.01,0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
x = cv2.line(image,(cols-1,righty),(0,lefty),(0,255,0),2)

In [8]:
def find_max_conts(contours):
    a = -1
    b = -1
    c = -1
    c1 = contours[0]
    c2 = contours[0]
    c3 = contours[0]
    for co in contours:
        l = cv2.arcLength(co,True)
        if l>a:
            c = b
            b = a
            a = l
            c3 = c2
            c2 = c1
            c1 = co
        elif l>b:
            c = b
            b = l
            c3 = c2
            c2 = co
        elif l>c:
            c = l
            c3 = co
    return c1,c2,c3

In [115]:
c1,c2,c3 = find_max_conts(contours)
c1.shape,c2.shape,c3.shape

((1267, 1, 2), (769, 1, 2), (333, 1, 2))

In [123]:
def extract_cont_features(cnt):
    """parameter is contour[i]
       output is Centroid(x,y),area,perimeter,angle,aspect ratio
    """
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt,True)
    rect = cv2.minAreaRect(cnt)
    angle = rect[2]
    asr = rect[1][0]/rect[1][1]
    return (cx,cy),area,perimeter,angle,asr

In [42]:
rect[1][0]/rect[1][1]

1.3671364921781812

In [124]:
def make_feature_table(contours):
    c1,c2,c3 = find_max_conts(contours)
    data = pd.DataFrame()
    for c in (c1,c2,c3):
        (x,y),a,p,an,ar = extract_cont_features(c)
        df = pd.DataFrame(np.array([x,y,a,p,an,ar]).reshape(1,6),columns=['Cx','Cy','Area','Perimeter','Angle','Aspect Ratio'])
        data = pd.concat([data,df],axis=1)
    return data 

In [125]:
d = make_feature_table(contours)

In [126]:
d

,Cx,Cy,Area,Perimeter,Angle,Aspect Ratio,Cx,Cy,Area,Perimeter,Angle,Aspect Ratio,Cx,Cy,Area,Perimeter,Angle,Aspect Ratio
0,298.0,260.0,96704.5,2912.667292,-5.219265,1.367136,303.0,246.0,83566.5,1672.177836,-71.864502,1.623663,262.0,387.0,1564.5,734.825462,-13.840694,3.000917
